

what is also practical if you want to have a second look at the tweets as they are saved in twitter: advanced search tool of twitter: https://twitter.com/search-advanced?lang=en


# Import modules

In [1]:
# Base and Cleaning 
import pandas as pd
import numpy as np
from datetime import datetime
import sys

#Natural Language Processing (NLP)
import spacy     # check out 
from spacy.tokenizer import Tokenizer
import gensim
from gensim.corpora import Dictionary # check out 
from gensim import models
from gensim.models.ldamulticore import LdaMulticore
from gensim.models.coherencemodel import CoherenceModel
from gensim.parsing.preprocessing import STOPWORDS as SW
from wordcloud import STOPWORDS # what is that? 
stopwords = set(STOPWORDS)

#Visualizations
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt 
!pip install pyLDAvis
import pyLDAvis.gensim_models
!pip install chart_studio
import chart_studio
import chart_studio.plotly as py 
import chart_studio.tools as tls
import warnings
warnings.simplefilter('ignore')#"error", "ignore", "always", "default", "module"


# settings for easier coding and inspection of data  
%matplotlib notebook
pd.set_option('display.max_colwidth', -1)

     |████████████████████████████████| 1.7 MB 5.1 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
  Created wheel for pyLDAvis: filename=pyLDAvis-3.3.1-py2.py3-none-any.whl size=136898 sha256=256d6bc84669dd7f94ed24407867b27e3593f8af892779b1aeff8d2937368a2d
  Stored in directory: /root/.cache/pip/wheels/c9/21/f6/17bcf2667e8a68532ba2fbf6d5c72fdf4c7f7d9abfa4852d2f
Successfully built pyLDAvis


/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable


     |████████████████████████████████| 64 kB 1.9 MB/s 
  Created wheel for retrying: filename=retrying-1.3.3-py3-none-any.whl size=11447 sha256=29828508d58dc456e848b3142c6080e69954a6a852c005b9604b2444c1161e4f
  Stored in directory: /root/.cache/pip/wheels/f9/8d/8d/f6af3f7f9eea3553bc2fe6d53e4b287dad18b06a861ac56ddf
Successfully built retrying


# Import data 

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
df = pd.read_csv("/content/drive/MyDrive/Ensemble Learning/Tweets/cyberbullying_tweets.csv")
df.head()
                             


,tweet_text,cyberbullying_type
0,"In other words #katandandre, your food was crapilicious! #mkr",not_cyberbullying
1,Why is #aussietv so white? #MKR #theblock #ImACelebrityAU #today #sunrise #studio10 #Neighbours #WonderlandTen #etc,not_cyberbullying
2,@XochitlSuckkks a classy whore? Or more red velvet cupcakes?,not_cyberbullying
3,"@Jason_Gio meh. :P thanks for the heads up, but not too concerned about another angry dude on twitter.",not_cyberbullying
4,"@RudhoeEnglish This is an ISIS account pretending to be a Kurdish account. Like Islam, it is all lies.",not_cyberbullying


# EDA - Exploratory Data Analysis 


In [4]:
# eda has also been performed in excel 
temp = df.sample(5)
[print(tweet,"\n") for tweet in temp.tweet_text.values]

These girls on MKR are giving me the shits! #MKR 

Rumor has it that bull Kareem is "on my head". I thought he was over that ass whippin i gave him. Fuck it. He kno's where I live. 

@GentlerMouse uh. It does save money. Eating things with a decent nutritional value is pretty expensive. Also other reasons. 

ISIS was originated, funded, armed &amp; supplied by Barack Obama &amp; Hillary Clinton &amp; promoted by John McCain, Lindsey Graham &amp; John Kerry, to create chaos in Egypt &amp; the Middle East. ISIS are terrorists~ but they also burn &amp; destroy the Quran &amp; mosques, proving they're not Muslims. 

@FrozenWes i hate numpads. 



[None, None, None, None, None]

# Preprocessing - 01 Cleaning of the text 

In [5]:
!pip install emoji
!pip install regex
import emoji  # needs to be installed with pip
import regex # needs to be installed with pip
import re
import string
from collections import Counter

     |████████████████████████████████| 174 kB 5.0 MB/s 
  Created wheel for emoji: filename=emoji-1.6.3-py3-none-any.whl size=170298 sha256=9cb285b775546cc00075ab00392432d3e9ce30340f4deb434769e5921127634f
  Stored in directory: /root/.cache/pip/wheels/03/8b/d7/ad579fbef83c287215c0caab60fb0ae0f30c4d7ce5f580eade
Successfully built emoji


In [6]:
def give_emoji_free_text(text):
    """
    Removes emoji's from tweets
    Accepts:
        Text (tweets)
    Returns:
        Text (emoji free tweets)
    """
    emoji_list = [c for c in text if c in emoji.UNICODE_EMOJI]
    if not emoji_list==[] :
        print("",emoji_list,end="")
    clean_text = ' '.join([str for str in text.split() if not any(i in str for i in emoji_list)])
    return clean_text

def url_free_text(text):   
    '''
    Cleans text from urls
    '''
    text, _ = re.subn(r'http\S+', '', text)
    return text 

def url_free_text_s(text):  # as some https were missed out on by the previous function
    '''
    Cleans text from urls
    '''
    text, _ = re.subn(r'https\S+', '', text)
    return text  


# Apply the function above and get tweets free of emoji's
call_emoji_free = lambda x: give_emoji_free_text(x)

In [7]:
# Apply `call_emoji_free` which calls the function to remove all emoji's
# Apply 'url_free_text'which calls the function to remove all links

pd.options.mode.chained_assignment = None  # default='warn' turn off one the warning ... 

for label in df.cyberbullying_type.unique():
    temp_df = df[df.cyberbullying_type ==label]
    print(label,"s tweets included the following emojis :")
    temp_df.tweet_text = temp_df.tweet_text.apply(call_emoji_free)
    print("\n")
    temp_df.tweet_text =temp_df.tweet_text.apply(url_free_text) 
    temp_df.tweet_text =temp_df.tweet_text.apply(url_free_text_s)
    df[df.cyberbullying_type ==label] = temp_df
    
df_prep=df # df_prep = result of preprocessing steps

not_cyberbullying s tweets included the following emojis :


gender s tweets included the following emojis :


religion s tweets included the following emojis :


other_cyberbullying s tweets included the following emojis :


age s tweets included the following emojis :


ethnicity s tweets included the following emojis :




### There are no emojis

# Tokenization


#pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_md-2.0.0/en_core_web_md-2.0.0.tar.gz



In [8]:
# Load spacy
import spacy
import en_core_web_sm
#nlp = spacy.load('en_core_web_sm')
nlp = en_core_web_sm.load()

In [9]:
# Tokenizer
tokenizer = Tokenizer(nlp.vocab)

# Custom stopwords
custom_stopwords = ['hi','\n','\n\n', '&amp;', ' ', '.', '-', 'got', 
                    "it's", 'it’s', "i'm", 'i’m', 'im', 'want', 'like', '$', '@']


# Customize stop words by adding to the default list
STOP_WORDS = nlp.Defaults.stop_words.union(custom_stopwords)

# ALL_STOP_WORDS = spacy + gensim + wordcloud 
ALL_STOP_WORDS = STOP_WORDS.union(SW).union(stopwords)


In [10]:
n = 10
print("These are",n,"of our stopwords:\n")
for i in list(ALL_STOP_WORDS)[:n]:
    print(i)

These are 10 of our stopwords:

wherein
’ve
she
rather
eight
what's
perhaps
since
i've
we're


In [11]:
tokens = []

for doc in tokenizer.pipe(df_prep.tweet_text,batch_size=500):
    doc_tokens = []    
    for token in doc: 
        if token.text.lower() not in ALL_STOP_WORDS:   #in the code that I was following, they only had stop_words .... 
            doc_tokens.append(token.text.lower())   
    tokens.append(doc_tokens)
    
# Makes tokens column
df_prep['tokens'] = tokens

In [12]:
df_prep.sample(2)

,tweet_text,cyberbullying_type,tokens
43064,fuck you and the projects... dumb ass niggers,ethnicity,"[fuck, projects..., dumb, ass, niggers]"
36638,"Big Boobs. A Thread. I had breasts before most of the girls in the locker room in 6th grade, and was bullied; I was groped in 8th grade, and by high school, I wore hoodies so people would leave me alone. That was a them problem that I internalized, falsely believing people would",age,"[big, boobs., thread., breasts, girls, locker, room, 6th, grade,, bullied;, groped, 8th, grade,, high, school,, wore, hoodies, people, leave, alone., problem, internalized,, falsely, believing, people]"


# Lemmatization and Synonyms

In [13]:
# more help: https://spacy.io/api/token
df_prep['tokens_back_to_text'] = [' '.join(map(str, l)) for l in df_prep['tokens']]


# to remove weird characters and valueless words (in sense of our analysis),
# we want to remove words with len<=2
# to keep interesting words with this length, we define the following list

list_useful_2char_word = ["uk","us","ad"] 


def get_lemmas(text):
    '''Used to lemmatize the processed tweets'''
    lemmas = []
    
    doc = nlp(text)
    for token in doc: 
        if ((token.is_stop == False) and (token.is_punct == False) and 
                (token.pos_!= 'PRON') and (token.is_digit== False) and 
                ((len(token)>2) or (token.text in list_useful_2char_word))): 
            lemmas.append(token.lemma_)
    
    return lemmas



In [14]:
# important this cell might take some minutes to run
# appllication of get_lemmas
df_prep['lemmas'] = df_prep['tokens_back_to_text'].apply(get_lemmas)


In [15]:
df_prep.sample(4) # visual inspection of lemmas

,tweet_text,cyberbullying_type,tokens,tokens_back_to_text,lemmas
143,"I'm much happier as a financially secure adult than as a kid in a paycheck to paycheck household (and I had great parents, just no $$$)",not_cyberbullying,"[happier, financially, secure, adult, kid, paycheck, paycheck, household, (and, great, parents,, $$$)]","happier financially secure adult kid paycheck paycheck household (and great parents, $$$)","[happy, financially, secure, adult, kid, paycheck, paycheck, household, great, parent]"
28105,It was time for Lavar Ball to shut up like a month or two ago...now he's just being an idiot,other_cyberbullying,"[time, lavar, ball, shut, month, ago...now, idiot]",time lavar ball shut month ago...now idiot,"[time, lavar, ball, shut, month, ago, idiot]"
19028,"Judeo Christian beliefs dominate the right - atheists support all efforts against that, including their unholy alliance with Islam. It’s why Islam isn’t called out by radical feminists for their its hate crimes. They’re allies against Jews and Christians. That’s all.",religion,"[judeo, christian, beliefs, dominate, right, atheists, support, efforts, that,, including, unholy, alliance, islam., islam, isn’t, called, radical, feminists, hate, crimes., they’re, allies, jews, christians., that’s, all.]","judeo christian beliefs dominate right atheists support efforts that, including unholy alliance islam. islam isn’t called radical feminists hate crimes. they’re allies jews christians. that’s all.","[judeo, christian, belief, dominate, right, atheist, support, effort, include, unholy, alliance, islam, islam, call, radical, feminist, hate, crime, ally, jews, christians]"
4505,Lmfao! @starrawassfuck pull out with somee kool-aid! #blackstatus love this girl always mkes me laugh!,not_cyberbullying,"[lmfao!, @starrawassfuck, pull, somee, kool-aid!, #blackstatus, love, girl, mkes, laugh!]",lmfao! @starrawassfuck pull somee kool-aid! #blackstatus love girl mkes laugh!,"[lmfao, @starrawassfuck, pull, somee, kool, aid, blackstatus, love, girl, mke, laugh]"


In [16]:
## check if we still got tokens with length <= 2 that are not specified in out list
def too_short_lemmas(x):
    for single_lemma in x:
        if (len(single_lemma)<=2) and (single_lemma not in list_useful_2char_word):
                print(single_lemma,end=",")
                

def check_for_min_length_lemma(x):
    too_short=False
    for single_lemma in x:
        if (len(single_lemma)<=2) and (single_lemma not in list_useful_2char_word):
            too_short = True
    return too_short


print("There are still",
      len(df_prep[df_prep.lemmas.apply(lambda x:check_for_min_length_lemma(x))]),
         "tweets with lemmas of length <=2 that are not allowed.",
     "\nThese too short lemmas are:\n")

# check if these lemmas can be removed or if they include important information
print(df_prep.lemmas.apply(lambda x:too_short_lemmas(x)).values) 


# this happened because the token was for example going --> len(going)>2 but lemma = go

There are still 2445 tweets with lemmas of length <=2 that are not allowed. 
These too short lemmas are:

go,go,go,go,go,go,go,go,go,go,go,ff,go,go,ff,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,lb,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,10,go,go,go,go,ff,go,up,go,go,do,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,op,go,go,go,go,go,ff,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,do,go,go,go,go,go,go,go,go,do,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,ht,go,go,go,go,do,go,go,go,go,go,go,go,go,go,do,go,go,go,go,go,go,pc,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,do,go,go,go,go,go,go,go,go,go,go,go,go,no,go,go,go,go,go,go,go,go,go,go,go,go,go,go,re,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,aw,go,go,go,go,go,go,go,go,go,go,go,go,

In [17]:
def remove_too_short(x):
    for single_lemma in x:
        if (len(single_lemma)<=2) and (single_lemma not in list_useful_2char_word):
                x.remove(single_lemma)
    return x

df_prep.lemmas = df_prep.lemmas.apply(lambda x:remove_too_short(x))

print("There are still",
      len(df_prep[df_prep.lemmas.apply(lambda x:check_for_min_length_lemma(x))]),
         "tweets with lemmas of length <=2 that are not allowed.")
# because of the small amount the effort to catch these slips was rated to high

There are still 6 tweets with lemmas of length <=2 that are not allowed.


In [18]:
# Make lemmas a string again
df_prep['lemmas_back_to_text'] = [' '.join(map(str,l)) for l in df_prep['lemmas']]


# handling of synonyms for topic based features
trump_synonyms = ["donald", "donald trump"] 
biden_synonyms = ["joe","joe biden"]
harris_synonyms = ["kamala","kamala harris"]
covid_synonyms = ["corona","covid-19","corona virus","virus","coronavirus"]

def replace_syn(repl_string,synonyms,col):
    for syn in synonyms:
        col_output = col.str.replace(syn,repl_string)
    return col_output
        
df_prep['lemmas_back_to_text'] = replace_syn("trump",trump_synonyms,df_prep['lemmas_back_to_text'])
df_prep['lemmas_back_to_text'] = replace_syn("biden",biden_synonyms,df_prep['lemmas_back_to_text'])
df_prep['lemmas_back_to_text'] = replace_syn("harris",harris_synonyms,df_prep['lemmas_back_to_text'])
df_prep['lemmas_back_to_text'] = replace_syn("covid",covid_synonyms,df_prep['lemmas_back_to_text'])


In [ ]:
df_prep.sample(2)

,tweet_text,cyberbullying_type,tokens,tokens_back_to_text,lemmas,lemmas_back_to_text
11649,tw/// rape if PS4 oomf doesn’t tell his fucking friend to stop making rape jokes and gay jokes I’m about to snap,gender,"[tw///, rape, ps4, oomf, doesn’t, tell, fucking, friend, stop, making, rape, jokes, gay, jokes, snap]",tw/// rape ps4 oomf doesn’t tell fucking friend stop making rape jokes gay jokes snap,"[tw///, rape, ps4, oomf, tell, fucking, friend, stop, make, rape, joke, gay, joke, snap]",tw/// rape ps4 oomf tell fucking friend stop make rape joke gay joke snap
44388,fucking fraud jesus man fuck that dumb nigger fucking creddies reckless for a gold i cant its so sad,ethnicity,"[fucking, fraud, jesus, man, fuck, dumb, nigger, fucking, creddies, reckless, gold, sad]",fucking fraud jesus man fuck dumb nigger fucking creddies reckless gold sad,"[fucking, fraud, jesus, man, fuck, dumb, nigger, fucking, creddie, reckless, gold, sad]",fucking fraud jesus man fuck dumb nigger fucking creddie reckless gold sad


In [19]:
lemma_tokens_all = []
# Tokenizer function
def tokenize(text):
    """
    Parses a string into a list of semantic units (words)
    Args:
        text (str): The string that the function will tokenize.
    Returns:
        list: tokens parsed out
    """
    # first of the following does not seem to work - or is this bcs of string digits
    tokens = re.sub(r'[^a-zA-Z 0-9]', '', text) 
    tokens = re.sub(r'[%s]' % re.escape(string.punctuation), '', text) # Remove punctuation
    tokens = re.sub(r'\w*\d\w*', '', text) # Remove words containing numbers
    tokens = re.sub(r'@*!*\$*', '', text) # Remove @ ! $
    tokens = tokens.strip(',') 
    tokens = tokens.strip('?') 
    tokens = tokens.strip('!') 
    tokens = tokens.strip("'") 
    tokens = tokens.strip(".") 


    tokens = tokens.lower().split() # Make text lowercase and split it
                
    lemma_tokens_all.extend(tokens)
    return tokens



In [20]:
# Apply tokenizer to df_all_orgas_cut_off and parallel create a list for every agency
# witl all lemma tokens

temp_series = pd.Series()
for i in df_prep.cyberbullying_type.unique():
    temp_df_part= df_prep[df_prep.cyberbullying_type==i]
    exec(f'list_{i}=[]') # create list to store all agency specific lemma tokens
    temp_df_part['lemma_tokens'] = temp_df_part['lemmas_back_to_text'].apply(tokenize)
    exec(f'list_{i}.extend(lemma_tokens_all)')
    lemma_tokens_all=[]
    temp_series= temp_series.append(temp_df_part['lemma_tokens'])

df_prep['lemma_tokens'] =temp_series

In [21]:
df_prep.head(2)

,tweet_text,cyberbullying_type,tokens,tokens_back_to_text,lemmas,lemmas_back_to_text,lemma_tokens
0,"In other words #katandandre, your food was crapilicious! #mkr",not_cyberbullying,"[words, #katandandre,, food, crapilicious!, #mkr]","words #katandandre, food crapilicious! #mkr","[word, katandandre, food, crapilicious, mkr]",word katandandre food crapilicious mkr,"[word, katandandre, food, crapilicious, mkr]"
1,Why is #aussietv so white? #MKR #theblock #ImACelebrityAU #today #sunrise #studio10 #Neighbours #WonderlandTen #etc,not_cyberbullying,"[#aussietv, white?, #mkr, #theblock, #imacelebrityau, #today, #sunrise, #studio10, #neighbours, #wonderlandten, #etc]",#aussietv white? #mkr #theblock #imacelebrityau #today #sunrise #studio10 #neighbours #wonderlandten #etc,"[aussietv, white, mkr, theblock, imacelebrityau, today, sunrise, studio10, neighbour, wonderlandten, etc]",aussietv white mkr theblock imacelebrityau today sunrise studio10 neighbour wonderlandten etc,"[aussietv, white, mkr, theblock, imacelebrityau, today, sunrise, studio10, neighbour, wonderlandten, etc]"


In [48]:
df_prep['lemma_tokens']
bigrams = []
for tweet in list(df_prep['lemma_tokens']):
  if len(tweet) == 0: continue
  previous_word = tweet[0]
  for word in tweet[1:]:
    bigrams.append([previous_word + ' ' + word])
    previous_word = word

In [51]:
gensim_dict2 = Dictionary(bigrams)

In [67]:
from gensim.models import Phrases
from gensim.models.phrases import Phraser
from gensim import models



docs = list(df_prep['lemmas_back_to_text'])

token_ = [doc.split(" ") for doc in docs]
bigram = Phrases(token_, min_count=1, threshold=2,delimiter=b' ')


bigram_phraser = Phraser(bigram)

bigram_token = []
for sent in token_:
    bigram_token.append(bigram_phraser[sent])

In [71]:
dict_ = gensim.corpora.Dictionary(bigram_token)

#print(dict_.token2id)
#Convert the word into vector, and now you can use tfidf model from gensim 
corpus = [dict_.doc2bow(text) for text in bigram_token]

tfidf_model = models.TfidfModel(corpus)

In [22]:
# inspect some lemma tokens for one of the classes
n=40
print("first",n,"lemma tokens of age bullying:")
print(*[i for i in list_age[:n]])

first 40 lemma tokens of age bullying:
home neighbor pick family mind son autistic feel high school name attack reason bully time step porch bully school high achieve boy use strategy maintain masculinity example bully disrupt lesson devalue girl achievement girl class 6th grade little autistic parent


In [46]:
# check how many tokens are left that do not comply to our length rule (if the number is small we keep them)
counter_01=0
for i in df_prep['lemma_tokens']:
    for k in i: 
        if (k.isnumeric()) or (len(k)<3) and (k  not in list_useful_2char_word):
            counter_01 +=1
            a=k
            #print(k,"is of type",type(k),"and found in:",i,"\n")
print("There are only",counter_01,"lemma tokens left, that do not comply to our length rule")


There are only 83 lemma tokens left, that do not comply to our length rule



#  create id2word and pre-process further 




In [24]:
# Create a gensim dictionary out of ALL lemma tokens
gensim_dict = Dictionary(df_prep['lemma_tokens'])


In [64]:
def some_stats_on_gen_dict(gd):
    n=60
    #This module implements the concept of a Dictionary 
    # – a mapping between words and their integer ids.
    print("The dict was created by lookint at",gd.num_docs,
          "tweets \nIt contains",len(gd),
          "different lemma tokens \n\nHere are the first",n,"tweets:")
    for i in range(n):
        print (gd.get(i),end=", ")

some_stats_on_gen_dict(gensim_dict2)



The dict was created by lookint at 486869 tweets 
It contains 308450 different lemma tokens 

Here are the first 60 tweets:
word katandandre, katandandre food, food crapilicious, crapilicious mkr, aussietv white, white mkr, mkr theblock, theblock imacelebrityau, imacelebrityau today, today sunrise, sunrise studio10, studio10 neighbour, neighbour wonderlandten, wonderlandten etc, xochitlsuckkks classy, classy whore, whore red, red velvet, velvet cupcake, jason_gio meh, meh thank, thank head, head concerned, concerned angry, angry dude, dude twitter, rudhoeenglish isis, isis account, account pretend, pretend kurdish, kurdish account, account islam, islam lie, raja5aab quickieleak, quickieleak yes, yes test, test god, god good, good bad, bad indifferent, indifferent weird, weird prove, prove god, god existence, itu sekolah, sekolah bukan, bukan tempat, tempat bully, bully jauh, jauh kaya, kaya neraka, karma hope, hope bite, bite kat, kat butt, butt nasty, nasty mkr, stockputout priest, re

In [62]:
gensim_dict2

In [60]:
# create a corpus with all final lemma tokens for each class
lemma_tokens_all = []
for sublist in df_prep.cyberbullying_type.unique():
    print(sublist)
    exec(f'lemma_tokens_all.append(list_{sublist})')

#print(lemma_tokens_all) 

corpus_all = [gensim_dict.doc2bow(d) for d in lemma_tokens_all]

# create work frequency list for all agency 
word_frequencies = [[(gensim_dict2[id], frequence) for id, 
                     frequence in couple] for couple in corpus_all]

not_cyberbullying
gender
religion
other_cyberbullying
age
ethnicity


# The following code creates the feature set for the bag of words approach

In [ ]:
word_frequencies_sorted = []
for tuples in word_frequencies:
    word_frequencies_sorted.append(sorted(tuples, key=lambda tup: tup[1], reverse = True))


In [ ]:
k = []
for word_frequencies_agency in word_frequencies_sorted:
    cumu_count = [0]
    counter = 1
    for i in word_frequencies_agency:
        cumu_count.append(cumu_count[counter-1]+i[1])
        counter+=1

    num_element = 0
    
    for i in cumu_count:
        if i > 0.2 * max(cumu_count):
            k.append(num_element)
            break
        num_element += 1

print(k)

[65, 7, 21, 59, 3, 5]


In [ ]:
bullying_type = 0
feature_list = []
for j in k:
    for i in range(0,j):
        feature_list.append(word_frequencies_sorted[bullying_type][i][0])
    bullying_type +=1
    

unique_feature_list = list(set(feature_list))
print("Number of words across all bullying_type:",len(feature_list))
print("Number of unique words across all bullying_type:",len(unique_feature_list))

Number of words across all bullying_type: 160
Number of unique words across all bullying_type: 101


In [ ]:
unique_feature_list

['kill',
 'see',
 'andre',
 'right',
 'people',
 'pretty',
 'work',
 'twitter',
 'look',
 'watch',
 'fucking',
 'home',
 'thing',
 'islam',
 'year',
 'love',
 'guy',
 'yeah',
 'thank',
 'mean',
 'person',
 'way',
 'woman',
 'bitch',
 'fuck',
 'religion',
 'ass',
 'need',
 'black',
 'know',
 'christian',
 'mkr2015',
 'girl',
 'tell',
 'tweet',
 'lol',
 'terrorism',
 'say',
 'think',
 'shit',
 'call',
 'dumb',
 'team',
 'support',
 'islamic',
 'muslims',
 'hope',
 'bully',
 'cook',
 'talk',
 'joke',
 'sure',
 'isis',
 'new',
 'bullying',
 'quran',
 'read',
 'day',
 'use',
 'nigger',
 'feminazi',
 'colin',
 'stop',
 'try',
 'high',
 'time',
 'daesh',
 'country',
 'class',
 'hate',
 'start',
 'help',
 'harassment',
 'yes',
 'gamergate',
 'que',
 'today',
 'freebsdgirl',
 'make',
 'female',
 'school',
 'happen',
 'terrorist',
 'dude',
 'come',
 'man',
 'blameonenotall',
 'kat',
 'idiot',
 'get',
 'feel',
 'muslim',
 'gay',
 'round',
 'good',
 'radical',
 'bad',
 'mkr',
 'rape',
 'stupid',
 

In [ ]:
dictOfWords = dict.fromkeys(unique_feature_list)
key_names = []
counter = 0
for key in dictOfWords:
    key_names.append(key)
    dictOfWords[key] = counter 
    counter+=1

lemma_list = list(df_prep.lemma_tokens)

len(lemma_list)
key_names.append('Not found')

In [ ]:
feature_matrix = []
feature_size = len(unique_feature_list) + 1

lemma_list

for array in lemma_list:
    temp = [0]*feature_size
    for k in array:
        index = dictOfWords.get(k,feature_size-1)
        temp[index] = 1
    
    feature_matrix.append(temp)

print(feature_size)
print(len(feature_matrix))

102
47692


In [ ]:
feature_df = pd.DataFrame(feature_matrix, index = df_prep.index, columns = key_names)
feature_df = feature_df.drop(['Not found'], axis=1)


In [ ]:
feature_df.head(2)

,kill,see,andre,right,people,pretty,work,twitter,look,watch,fucking,home,thing,islam,year,love,guy,yeah,thank,mean,person,way,woman,bitch,fuck,religion,ass,need,black,know,christian,mkr2015,girl,tell,tweet,lol,terrorism,say,think,shit,...,colin,stop,try,high,time,daesh,country,class,hate,start,help,harassment,yes,gamergate,que,today,freebsdgirl,make,female,school,happen,terrorist,dude,come,man,blameonenotall,kat,idiot,get,feel,muslim,gay,round,good,radical,bad,mkr,rape,stupid,lot
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0


In [ ]:
combined_features = pd.concat([df_prep,feature_df],axis = 1)
combined_features.head(2)


,tweet_text,cyberbullying_type,tokens,tokens_back_to_text,lemmas,lemmas_back_to_text,lemma_tokens,kill,see,andre,right,people,pretty,work,twitter,look,watch,fucking,home,thing,islam,year,love,guy,yeah,thank,mean,person,way,woman,bitch,fuck,religion,ass,need,black,know,christian,mkr2015,girl,...,colin,stop,try,high,time,daesh,country,class,hate,start,help,harassment,yes,gamergate,que,today,freebsdgirl,make,female,school,happen,terrorist,dude,come,man,blameonenotall,kat,idiot,get,feel,muslim,gay,round,good,radical,bad,mkr,rape,stupid,lot
0,"In other words #katandandre, your food was crapilicious! #mkr",not_cyberbullying,"[words, #katandandre,, food, crapilicious!, #mkr]","words #katandandre, food crapilicious! #mkr","[word, katandandre, food, crapilicious, mkr]",word katandandre food crapilicious mkr,"[word, katandandre, food, crapilicious, mkr]",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
1,Why is #aussietv so white? #MKR #theblock #ImACelebrityAU #today #sunrise #studio10 #Neighbours #WonderlandTen #etc,not_cyberbullying,"[#aussietv, white?, #mkr, #theblock, #imacelebrityau, #today, #sunrise, #studio10, #neighbours, #wonderlandten, #etc]",#aussietv white? #mkr #theblock #imacelebrityau #today #sunrise #studio10 #neighbours #wonderlandten #etc,"[aussietv, white, mkr, theblock, imacelebrityau, today, sunrise, studio10, neighbour, wonderlandten, etc]",aussietv white mkr theblock imacelebrityau today sunrise studio10 neighbour wonderlandten etc,"[aussietv, white, mkr, theblock, imacelebrityau, today, sunrise, studio10, neighbour, wonderlandten, etc]",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0


In [ ]:
# test whether the feature creation works

## entry number to test
test = 899
print(combined_features.iloc[test][6])
feature_df.iloc[test][feature_df.iloc[test]>0]


['halalflaws', 'greenlinerzjm', 'biebervalue', 'problem', 'force', 'wear', 'microbrain']


Series([], Name: 899, dtype: int64)

In [ ]:
assert len(combined_features) == len(df) 

In [ ]:
## save file to drive
combined_features.to_csv("/content/drive/MyDrive/Ensemble Learning/bag of words.csv")